In [71]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Exploration du dataset
- Aperçu général: colonnes, valeurs manquantes, duplicats, types de données
- Vérification et nettoyage minimal (si nécéssaire)

In [72]:
# |exporti
import streamlit as st
import pandas as pd
from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

st.title("Analyse des contenus Netflix")

st.header("Exploration du dataset")

df = pd.read_csv("data/netflix_titles.csv")

# Afficher les colonnes
st.subheader("Colonnes :")
st.write(", ".join([c.title() for c in df.columns]))

# Compter les valeurs manquantes par colonne
st.subheader("Valeurs manquantes par colonne :")
st.write(df.isna().sum())

# Afficher le nombre de duplcats
st.subheader("\nNombre de doublons :")
st.write(df.duplicated().sum())

# Afficher les types de données
st.subheader("\nTypes de données :")
st.write(df.dtypes.groupby(df.dtypes).size())

# Nettoyer les données en supprimant les doublons
df_cleaned = df.drop_duplicates()
st.write("\nDonnées nettoyées, doublons supprimés.")

# Analyse des contenus Netflix

## Exploration du dataset

### Colonnes :

Show_Id, Type, Title, Director, Cast, Country, Date_Added, Release_Year, Rating, Duration, Listed_In, Description

### Valeurs manquantes par colonne :

...

### 
Nombre de doublons :

np.int64(0)

### 
Types de données :

int64      1
object    11
dtype: int64


Données nettoyées, doublons supprimés.

# Analyse des contenus
- Films vs séries : proportions, tendances par année.
- Genres principaux (listed_in) : regroupement, fréquences.
- Répartition géographique (country).
- Casting & réalisateurs : analyse simple (comptages, noms fréquents).

In [73]:
# |exporti

import pandas as pd

from streamlit_jupyter import StreamlitPatcher, tqdm

StreamlitPatcher().jupyter()

st.header("Analyse des contenus")

pd.set_option('display.max_rows', None)
df = pd.read_csv("data/netflix_titles.csv")

# Afficher le nombre total de films et son pourcentage
total_movies = df[df["type"] == "Movie"]
st.subheader("Nombre total de films :")
st.write(len(total_movies), " (", round(len(total_movies) / len(df) * 100, 2), "%)")

# Afficher le nombre total de séries et son pourcentage
total_series = df[df["type"] == "TV Show"]
st.subheader("Nombre total de séries :")
st.write(len(total_series), " (", round(len(total_series) / len(df) * 100, 2), "%)")

# Afficher le nombre de films et séries par année de sortie
st.subheader("Nombre de films et séries par année de sortie :")
pivot_df = df.pivot_table(index='type', columns='release_year', values='show_id', aggfunc='count', fill_value=0)
st.write(pivot_df)

# Afficher le nombre de films et séries par catégorie
df_listed_in = df.dropna(subset=['listed_in']).copy()
df_listed_in['listed_in'] = df_listed_in['listed_in'].str.split(', ') # Séparer les catégories
df_exploded = df_listed_in.explode('listed_in') # Exploser les lignes pour chaque catégorie
category_type_counts = df_exploded.groupby(['listed_in', 'type'])['show_id'].count().unstack(fill_value=0) # Grouper et compter
st.subheader("Nombre de films et séries par catégorie :")
st.write(category_type_counts)

# Afficher le nombre de films et séries par pays
df_countries = df.dropna(subset=['country']).copy()
df_countries['country'] = df_countries['country'].str.split(', ') # Séparer les pays
df_exploded = df_countries.explode('country') # Exploser les lignes pour chaque pays
country_type_counts = df_exploded.groupby(['country', 'type'])['show_id'].count().unstack(fill_value=0) # Grouper et compter le nombre de films et séries par pays
st.subheader("Nombre de films et séries par pays (multi-pays comptés) :")
st.write(country_type_counts)

# Afficher les 10 acteurs les plus fréquents
df_cast = df.dropna(subset=['cast']).copy()
df_cast['cast'] = df_cast['cast'].str.split(', ') # Séparer les acteurs
df_exploded = df_cast.explode('cast') # Exploser les lignes pour chaque acteur
actor_counts = df_exploded['cast'].value_counts().head(10) # Compter les acteurs les plus fréquents
st.subheader("Les 10 acteurs les plus fréquents :")
st.write(actor_counts)

# Afficher les 10 réalisateurs les plus fréquents
df_directors = df.dropna(subset=['director']).copy()
df_directors['director'] = df_directors['director'].str.split(', ') # Séparer les réalisateurs
df_exploded = df_directors.explode('director') # Exploser les lignes pour chaque réalisateur
director_counts = df_exploded['director'].value_counts().head(10) # Compter les réalisateurs les plus fréquents
st.subheader("Les 10 réalisateurs les plus fréquents :")
st.write(director_counts)

## Analyse des contenus

### Nombre total de films :

6131

 (

69.62

...

type,Movie,TV Show
country,,
,1,1
Afghanistan,1,0
Albania,1,0
Algeria,3,0
Angola,1,0
Argentina,71,20
Armenia,1,0
Australia,94,66
Austria,11,1


### Les 10 acteurs les plus fréquents :

cast
Anupam Kher         43
Shah Rukh Khan      35
Julie Tejwani       33
Takahiro Sakurai    32
Naseeruddin Shah    32
Rupa Bhimani        31
Akshay Kumar        30
Om Puri             30
Yuki Kaji           29
Paresh Rawal        28
Name: count, dtype: int64

### Les 10 réalisateurs les plus fréquents :

director
Rajiv Chilaka          22
Jan Suter              21
Raúl Campos            19
Marcus Raboy           16
Suhas Kadav            16
Jay Karas              15
Cathy Garcia-Molina    13
Martin Scorsese        12
Jay Chapman            12
Youssef Chahine        12
Name: count, dtype: int64

# Analyse temporelle
- Distribution par année de sortie (release_year).
- Analyse de la colonne date_added.

# Visualisation
- Représentations obligatoires :
- histogrammes, countplots, boxplots (Seaborn / Matplotlib)
- visualisations interactives (Plotly)
- Choix libres : nuages de mots, diagrammes circulaires, timelines, etc.

# Synthèse
- Résumé des observations principales.
- Tendances remarquables (contenus récents, pays dominants, genres populaires).

# Exportation du notebook 

In [74]:
from nbdev.export import nb_export
nb_export('project.ipynb', lib_path='./app/', name='netflix_analysis')